<a href="https://colab.research.google.com/github/ronakdudhani/Covid-Knowledge-Graph/blob/main/Gemini_question_answer_connnection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gemini setup**

In [42]:
!pip install -q -U google-generativeai


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip


In [43]:
!wget https://raw.githubusercontent.com/Singhsansar/Covid-Knowledge-Graph/main/question.csv

E0103 15:55:51.556875686   28224 backup_poller.cc:127]                 Run client channel backup poller: UNKNOWN:pollset_work {created_time:"2024-01-03T15:55:51.556773907+00:00", children:[UNKNOWN:Bad file descriptor {syscall:"epoll_wait", os_error:"Bad file descriptor", errno:9, created_time:"2024-01-03T15:55:51.556721769+00:00"}]}
--2024-01-03 15:55:51--  https://raw.githubusercontent.com/Singhsansar/Covid-Knowledge-Graph/main/question.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 136061 (133K) [text/plain]
Saving to: ‘question.csv.1’

question.csv.1      100%[===================>] 132.87K  --.-KB/s    in 0.1s    

2024-01-03 15:55:52 (1.13 MB/s) - ‘question.csv.1’ saved [136061/136061]



In [44]:
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [45]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY='Enter your api key'
genai.configure(api_key=GOOGLE_API_KEY)

In [46]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-pro
models/gemini-pro-vision


In [47]:
model = genai.GenerativeModel('gemini-pro')

In [48]:
%%time
questions = 'who is prime minister on India?'
prompt = f'''
          instructions: consider writing the paragraphs no bullet points should be there , do not exceed 500 words, write all the answers in 200-400 words strictly follow this 
          question: {questions}
          answer:
          '''

response = model.generate_content(prompt)


CPU times: user 7.02 ms, sys: 3.96 ms, total: 11 ms
Wall time: 11.8 s


In [49]:
to_markdown(response.text)


> India, a nation brimming with vibrancy, diversity, and a rich tapestry of cultures, is led by its Prime Minister, an individual entrusted with the immense responsibility of guiding the country towards progress and prosperity. The Prime Minister of India holds the highest executive office in the Government of India and serves as the head of the Council of Ministers. This esteemed position requires a leader of exceptional vision, strategic thinking, and the ability to navigate the complexities of a nation as vast and dynamic as India.
> 
> The journey to becoming Prime Minister of India is an arduous one, requiring years of dedicated service, unwavering commitment, and a deep understanding of the country's intricate social, economic, and political landscape. The path typically begins with active involvement in politics at the state or local level, where aspiring leaders gain firsthand experience in governance and public service. As they rise through the ranks, they cultivate a strong network of supporters and allies who play a crucial role in their political ascent.
> 
> To become Prime Minister, a candidate must first secure a seat in the Lok Sabha, the lower house of the Parliament of India. The Lok Sabha is composed of 545 members, directly elected by the people of India through a first-past-the-post electoral system. Once elected, members of the Lok Sabha may nominate a candidate for the position of Prime Minister. The candidate who secures the support of a majority of the Lok Sabha members is appointed as the Prime Minister by the President of India.
> 
> The Prime Minister's responsibilities are vast and encompass a wide range of areas that impact the lives of all Indians. These include formulating and implementing policies related to economic development, social welfare, national security, foreign affairs, and environmental protection. The Prime Minister also oversees the functioning of various government ministries and departments, ensuring that they operate efficiently and effectively in pursuit of the nation's goals.
> 
> Effective leadership at the helm of the Government of India requires a combination of political acumen, administrative skills, and the ability to strike a balance between diverse interests and perspectives. The Prime Minister must be able to navigate the complex political landscape, forge consensus among various stakeholders, and make decisive decisions in times of crisis. Additionally, the Prime Minister serves as the face of India on the global stage, representing the country's interests and values in international forums and negotiations.
> 
> The role of Prime Minister of India is one of immense responsibility and profound significance. It demands a leader with a clear vision for the country's future, the courage to make tough decisions, and the unwavering dedication to serve the people of India. The Prime Minister is the custodian of the nation's aspirations and the architect of its progress, shaping the destiny of a vibrant and ever-evolving democracy.

# **Questions Setup**

In [52]:
import csv
import time
import requests
import pandas as pd
def store_data(data, filename):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['Question', 'Answer']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for entry in data:
                writer.writerow({'Question': entry['Question'], 'Answer': entry['Answer']})
    except Exception as e:
        print(f"Error writing to CSV: {e}")

In [53]:


df = pd.read_csv('question.csv')
df = df.iloc[:1000]
def make_api_call(question):
    try:
        prompt = f'''
          Context: Understanding the Impact of Coronavirus (COVID-19)
          Instructions: Please provide comprehensive answers within the range of 200-400 words. Strictly adhere to this word limit, avoid using bullet points, and ensure there are no gaps in the responses. Additionally, refrain from mentioning the questions in your replies.
          
          Given the ongoing global situation surrounding the coronavirus and COVID-19, share insights, information, and analyses related to the pandemic.
          question: {question}
          Answer:
          '''
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error making API call: {e}")
        return None


model = genai.GenerativeModel('gemini-pro')

answers = []
i=0
for index, row in df.iterrows():
    question = row.get('question', '')
    content = make_api_call(question)
    if content is not None:
        answers.append({'Question': question, 'Answer': content})
        print(f'{i}. {question}')
        i = i+1
        time.sleep(5)
    else:
        print(f"Skipping question '{question}' due to API error.")
        time.sleep(20)



0. Can pets catch the cold?
1. Is the Common Cold an Immune Overreaction?
2. Air purifier agains bacteria and viruses?
3. Why are bats the source of dangerous coronavirus pandemics?
4. How do bats survive their own coronaviruses?
5. Can Coronaviridae mutate into something as dangerous as Flioviridae?
6. Evolution history of virus tracing
7. Coronavirus origin could be bat, why this happen while people have been eating bats for centuries?
8. How exactly does alcohol solution kill or neutralize viruses?
9. What are the environmental conditions for SARS-CoV-2 to survive?
10. Why would a 2019-nCoV protein sequence in the NCBI database match a protein submitted in 2018?
11. How does a Coronavirus "test kit" work?
12. How are lipid-coated mRNA-based vaccines transported into cells for expression?
13. Why would it be dangerous to consume cooked bats even if they are linked to viruses?
14. how the coronavirus was distinguished?
15. What is the size (diameter) of the SARS-CoV-2 virus?
16. Does 

In [54]:

store_data(answers, 'Generated_data.csv')

In [55]:
import pandas as pd
data_g = pd.read_csv('Generated_data.csv')

In [61]:
data_g.head(20)

,Question,Answer
0,Can pets catch the cold?,"While pets can contract respiratory illnesses,..."
1,Is the Common Cold an Immune Overreaction?,"The common cold, a prevalent respiratory infec..."
2,Air purifier agains bacteria and viruses?,Air purifiers play a crucial role in combating...
3,Why are bats the source of dangerous coronavir...,Bats are a reservoir for a diverse array of co...
4,How do bats survive their own coronaviruses?,Bats are a unique group of mammals that play a...
5,Can Coronaviridae mutate into something as dan...,The Coronaviridae family encompasses a wide ra...
6,Evolution history of virus tracing,The evolution of viruses has been an ongoing p...
7,"Coronavirus origin could be bat, why this happ...",The exact origin of the coronavirus remains a ...
8,How exactly does alcohol solution kill or neut...,"Alcohol-based solutions, particularly those wi..."
9,What are the environmental conditions for SARS...,"The survival of SARS-CoV-2, the virus that cau..."
